<a href="https://colab.research.google.com/github/YonggunJung/Project1-KOSPI/blob/main/2_GDP%2C_GNI_%EC%A0%84%EC%B2%98%EB%A6%AC_%EB%85%84%EB%8F%84%EB%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 기본 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil

import re

import surprise

# 경고 뜨지 않게..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'Apple Gothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation

# CNN : 커널을 통해 합성곱을 구하는것, 이미지의 특정이 두드러지게 함
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D
# MaxPooling : 커널 내에서 가장 큰 값을 추출하는 방식으로 불필요 부분을 제거
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D
# Flatten : 다차원의 이미지 데이터를 선형 회귀 은닉층으로 전달 전에 1차원으로 변환
from tensorflow.keras.layers import Flatten
# Dropout : 이미지나 영상, 음파 등의 데이터는 오랫동안 학습을 진행 시켜야 함
# 하지만 너무 빨리 과적합이 발생되면 조기 중단 때문에 횟수가 줄어듬
# 이에 은닉의 노드를 일부 사용하지 않으므로써 과적합이 빨리 오는 것을 예방하고 오랫동안 학습이 가능케 함
from tensorflow.keras.layers import Dropout
# Embedding : 단어의 수 조정
from tensorflow.keras.layers import Embedding

# LSTM
from tensorflow.keras.layers import LSTM


# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 저장된 학습모델 복원
from tensorflow.keras.models import load_model

# 모델 자동 저장
from tensorflow.keras.callbacks import ModelCheckpoint
# 성능이 더이상 좋아지지 않을 경우 중단
from tensorflow.keras.callbacks import EarlyStopping

# 문장 자르기
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장 데이터의 단어 데이터의 수를 동일한 수로 맞춰줌
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 가지고 단어 사전을 만듦
from tensorflow.keras.preprocessing.text import text_to_word_sequence


# 평가 함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 -> 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검즘용으로 나누기
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


from surprise import SVD, Dataset, Reader, accuracy
from surprise import model_selection


from konlpy.tag import Okt



# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 있는데 이때 시드를 고정 시킴
random_seed = 1

np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU메모리 사용을 필요한 만큼만 쓸수 있다록 함
# 컴퓨터에 있는 GPU정도들을 가져온다.
gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면
if len(gpus) >0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# 기본
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래서 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic' # 맥 용
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 두개 이상 설정할 경우
from sklearn.model_selection import cross_validate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedGroupKFold

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV


# 평가함수 - 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 평가함수 - 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류용
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀용
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 시간 측정을 위한 모듈
import datetime

# 저장을 위한 모듈
import pickle


In [ ]:
# 1. GDP, GNI 불러오기
# 단위 : 백만달러
df10 = pd.read_csv('./data/GDPGNI년도.csv')
df10

,통계표,항목명1,단위,변환,1981,1982,1983,1984,1985,1986,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,9.1.4.2. 국제 주요국 국내총생산(GDP),한국,백만달러,원자료,"72,933.5","78,349.5","87,760.5","97,511.3","101,296.2","116,836.0",...,"1,253,289.5","1,278,046.6","1,370,633.0","1,484,488.5","1,466,038.9","1,499,679.8","1,623,074.1","1,725,373.4","1,651,422.9","1,637,895.8"
1,9.1.4.4. 국제 주요국 국민총소득(GNI),한국,백만달러,원자료,"71,939.7","77,625.0","86,845.3","96,132.0","99,075.0","114,324.6",...,"1,261,047.4","1,291,410.8","1,379,369.0","1,491,673.3","1,470,624.6","1,505,162.3","1,629,690.1","1,732,322.3","1,665,675.9","1,650,493.2"


In [ ]:
df10.drop(['통계표', '항목명1', '단위', '변환'], axis=1, inplace=True)
# df10.columns.str.replace(',', '')
df10

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"72,933.5","78,349.5","87,760.5","97,511.3","101,296.2","116,836.0","147,949.3","199,592.6","246,928.8","283,365.2",...,"1,253,289.5","1,278,046.6","1,370,633.0","1,484,488.5","1,466,038.9","1,499,679.8","1,623,074.1","1,725,373.4","1,651,422.9","1,637,895.8"
1,"71,939.7","77,625.0","86,845.3","96,132.0","99,075.0","114,324.6","146,157.4","198,046.8","246,202.5","283,081.3",...,"1,261,047.4","1,291,410.8","1,379,369.0","1,491,673.3","1,470,624.6","1,505,162.3","1,629,690.1","1,732,322.3","1,665,675.9","1,650,493.2"


In [ ]:
df10 = df10.T
df10

,0,1
1981,"72,933.5","71,939.7"
1982,"78,349.5","77,625.0"
1983,"87,760.5","86,845.3"
1984,"97,511.3","96,132.0"
1985,"101,296.2","99,075.0"
1986,"116,836.0","114,324.6"
1987,"147,949.3","146,157.4"
1988,"199,592.6","198,046.8"
1989,"246,928.8","246,202.5"
1990,"283,365.2","283,081.3"


In [ ]:
df10.columns = ['GDP', 'GNI']
df10

,GDP,GNI
1981,"72,933.5","71,939.7"
1982,"78,349.5","77,625.0"
1983,"87,760.5","86,845.3"
1984,"97,511.3","96,132.0"
1985,"101,296.2","99,075.0"
1986,"116,836.0","114,324.6"
1987,"147,949.3","146,157.4"
1988,"199,592.6","198,046.8"
1989,"246,928.8","246,202.5"
1990,"283,365.2","283,081.3"


In [ ]:
df10['GDP'] = df10['GDP'].str.replace(',', '').astype('float64')
df10['GNI'] = df10['GNI'].str.replace(',', '').astype('float64')
df10

,GDP,GNI
1981,72933.5,71939.7
1982,78349.5,77625.0
1983,87760.5,86845.3
1984,97511.3,96132.0
1985,101296.2,99075.0
1986,116836.0,114324.6
1987,147949.3,146157.4
1988,199592.6,198046.8
1989,246928.8,246202.5
1990,283365.2,283081.3


In [ ]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 1981 to 2020
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GDP     40 non-null     float64
 1   GNI     40 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


In [ ]:
df10.to_csv('./data/전처리/GDPGNI전처리.csv', encoding='utf-8-sig')